In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd
from itertools import permutations
from doublepath import first_path, nono_points, second_path
import seaborn as sns
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [ ]:
#the core of the greedy algorithm
def first_path(data):
    import numpy as np
    
    path = []
    distanza = []
    punto = 0
    inizio = int(input('Punto Iniziale : ', ))
    
    path.append(data[inizio][0])
    punto = data[inizio][1:]
    data = np.delete(data, inizio, 0)
    
    for i in range(len(data)):
        
        x = np.full(len(data), punto[0])
        y = np.full(len(data), punto[1])
        diff = np.abs(x - data[:,1]) + np.abs(y - data[:,2])
        
        ind = np.where(diff == min(diff))[0][0]
        
        path.append(int(data[ind][0]))
        distanza.append(diff[ind])
        
        punto = data[ind][1:]
        
        data = np.delete(data, ind, 0)
        
        if((i % 1000) == 0):
            print(i, sum(distanza))
            
    return path, distanza

#this function finds the points that should not be used in the second path
def excluded_points(path):
    
    points = []
    
    #trasformo in interi
    
    path = [int(x) for x in path]
    
    for i in range(len(path)):
        points.append([])
        ultimo = len(path) - 1
        if((i==0) or (i==ultimo)):
            if(i==0):
                points[i].append(path[i+1])
            else:
                points[i].append(path[i-1])
                
        else:
            points[i].append(path[i-1])
            points[i].append(path[i+1])
    
    
    return points

#this function compute the second path
def second_path(data, points, path_1):
    import numpy as np
    from timeit import default_timer as timer
    
    path = []
    distanza = []
    
    primo = np.array(path_1)
    random = int(input('Random second point ', ))
    
    #trovo l'indice a cui corrisponde quel numero
    path.append(data[:,0][random])
    
    coords = data[random][1:3]
    
    data = np.delete(data, random, 0)
    
    for i in range(len(data)):
        start = timer()
        #trovo i punti con cui non si deve collegare
        try:
            ind = np.where(primo == path[-1])[0][0]
        except:
            print('no points')
        
        #dalla lista points posso recuperare i punti 
        if(len(points[ind]) == 2):
            antecedente = points[ind][0]
            successivo = points[ind][1]
            
            #individuo con un try se i punti sono presenti nel dataset
            ant = np.where(data[:,0] == antecedente)[0]
            pre = np.where(data[:,0] == successivo)[0]
            
            #calcolo le differenze dal punto
            x = np.full(len(data), coords[0])
            y = np.full(len(data), coords[1])
            
            diff = np.abs(x - data[:,1]) + np.abs(y - data[:,2])
            
            #sempre con un try provo a toglierli dal vettore delle differenze
            diff[ant] = max(diff)
            diff[pre] = max(diff)
            
        else:
            unicum = points[ind][0]
            
            unic = np.where(data[:,0] == unicum)[0]
            
            x = np.full(len(data), coords[0])
            y = np.full(len(data), coords[1])
            
            diff = np.abs(x - data[:,1]) + np.abs(y - data[:,2])
            
            
            diff[unic] = max(diff)
            
        end = timer()
        tempo = end - start
        #ora posso estrapolare il minimo
        if((i % 1000) == 0 ):
            print(i, tempo, sum(distanza))
        else:
            pass
        
        new_min = np.where(diff == min(diff))[0][0]
        
        path.append(int(data[new_min][0]))
        distanza.append(diff[new_min])
        
        coords = data[new_min][1:3]
        
        data = np.delete(data, new_min, 0)
        
    return path

In [ ]:
data = pd.read_csv('santa_cities.csv')

#first clusterization with dbscan
model = DBSCAN(eps=180).fit(data[['x','y']])

data['labels'] = model.labels_

#all minor subclusters are put together with the largest ones
data['labels'].loc[data['labels'] == 1] = -1
data['labels'].loc[data['labels'] == 2] = 0

#applying dbscan on noisy points
campione = data[data['labels'] == -1].copy()
rumore = DBSCAN(eps=680).fit(campione[['x','y']])
campione['labels_2'] = rumore.labels_

#renaming the columns
campione = campione.drop('labels', axis=1)
campione.columns = ['id','x','y','labels']

#renaming the main cluster
rima = data[data['labels'] == 0].copy()
rima['labels'] = 3

#putting together all the clusters found in the decomposition
unico = pd.concat([campione, rima], axis=0)

#function to evaluate the distance in terms of Manhattan Distance
def dist_eval(dati, path):
    distanza = []
    
    for c in range(len(path) - 1):
        primo = np.where(dati[:,0] == path[c])[0][0]
        secondo = np.where(dati[:,0] == path[c+1])[0][0]

        x = dati[primo][1]
        x2 = dati[secondo][1]

        y = dati[primo][2]
        y2 = dati[secondo][2]

        man = np.abs(x - x2) + np.abs(y - y2)
        distanza.append(man)
        
    summing = sum(distanza)   
    
    return summing

#calculate all the possible permutations of the subpath in order to find the best one
def n_perm(percorso, piece, b, dati):
    if(b == 0):
        last = []
        quarto = []
        last.append(percorso[-1])
        quarto.append(percorso[4])
        
        to_evaluate = last + piece + quarto
        ##ora valuto la distanza di questo percorso
        distance = dist_eval(dati, to_evaluate)
        
        #una volta ottenuta la distanza di questo subpath devo valutare quale combinazione di piece
        #mi permette di migliorare la distanza totale del subpath
        ##--- calcolo quindi tutte le combinazioni possibili
        temp = set()
        #--------
        for perm in permutations(piece):
            temp.add(perm)
        temp = list(temp)
        
        distanza = []
        
        for z in range(len(temp)):
            to_evaluate[1:5] = temp[z]
            #valuto la nuova distanza
            hg = dist_eval(dati, to_evaluate)
            distanza.append(hg)
            
        distanza = np.array(distanza)
        ind_min = np.where(distanza == min(distanza))[0][0]
        
        best_combination = temp[ind_min]
    
    else:
        last = []
        quarto = []
        last.append(percorso[(b*4)-1])
        quarto.append(percorso[(b+1)*4])
        to_evaluate = last + piece + quarto
        distance = dist_eval(dati, to_evaluate)
        
        temp = set()
        #--------
        for perm in permutations(piece):
            temp.add(perm)
        temp = list(temp)
        
        distanza = []
        
        for z in range(len(temp)):
            to_evaluate[1:5] = temp[z]
            #valuto la distanza
            hg = dist_eval(dati, to_evaluate)
            distanza.append(hg)
            
        distanza = np.array(distanza)
        ind_min = np.where(distanza == min(distanza))[0][0]
        
        best_combination = temp[ind_min]
    
    return best_combination

#applying local 3 opt optimization
def three_opt(percorso, dati, k):
    
    #il 3 opt prende tre edges li spezza e li ricrea finchè non ha trovato una combinazione migliore
    ##per prima cosa estraggo gli id
    alternativo = percorso.copy()
    
    for a in range(k):
        
        for b in range(round(len(percorso) / 4) - 1):
            piece = alternativo[(b*4)+a:((b+1)*4)+a]
            #verifico che il frammento non sia vuoto
            if((bool(piece) == False) or (len(piece) < 4)):
                break
            else:
                print(b)
                if(b == 37499):
                    break
                ##calcolo tutte le permutazioni possibili del percorso e valuto la distanza del nuovo
                ##alla fine ottengo così la best combination per quel frammento di percorso
                new_comb = n_perm(percorso, piece, b, dati)
                
                #ora posso sostituire il nuovo pezzo nel percorso precedente
                alternativo[(b*4)+a:((b+1)*4)+a] = new_comb
        
                
    return alternativo

#iteration on all the clusters of the dataset

ordine = [0, 1, 3, 2]

path1 = []
path2 = []

for i in range(len(ordine)):
    #dataframe temporaneo
    temp = unico[unico['labels'] == ordine[i]].copy()
    temp = temp[['id','x','y']]
    temp = temp.reset_index().drop('index', axis=1)
    temp = temp.values
    
    #ora faccio il greedy
    temp_1 = first_path(temp)
    
    #ottimizzo
    new = three_opt(dati=temp, k=1, percorso=temp_1[0])
    
    nono = nono_points(path=new)
    
    temp_2 = second_path(data=temp, path_1=new, points=nono)
    
    path1 = path1 + new
    path2 = path2 + temp_2
    
    print(i, len(set(path1)), len(set(path2)))